# Acquire data
- read_clipboard()
- read_excel
- read_csv #from googlesheet, 

In [18]:
import pandas as pd
import numpy as np
import acquire
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

In [19]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

**Acquire**
**Data Summary**
- info
- describe
- head/tail
- value_counts
- shape
- isnull

In [20]:
df = pd.DataFrame(acquire.get_data_from_mysql())

Type in your queryselect * from passengers
Name the databasetitanic_db


In [4]:
df.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,None,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,None,Southampton,1
3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,0,3,male,35.0,0,0,8.0500,S,Third,None,Southampton,1


In [8]:
df.info()
# takeaway - deck contain way too many null

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
passenger_id    891 non-null int64
survived        891 non-null int64
pclass          891 non-null int64
sex             891 non-null object
age             714 non-null float64
sibsp           891 non-null int64
parch           891 non-null int64
fare            891 non-null float64
embarked        889 non-null object
class           891 non-null object
deck            203 non-null object
embark_town     889 non-null object
alone           891 non-null int64
dtypes: float64(2), int64(6), object(5)
memory usage: 90.6+ KB


In [11]:
df.isnull().sum()
# take away age, embarked/embarked_town need to be handled

passenger_id      0
survived          0
pclass            0
sex               0
age             177
sibsp             0
parch             0
fare              0
embarked          2
class             0
embark_town       2
alone             0
dtype: int64

In [19]:
# no empty space
# 177 NaN
# df.age.value_counts()


In [5]:
# no empty space
df.embarked.value_counts()

S    644
C    168
Q     77
Name: embarked, dtype: int64

Prepare Data

- drop columns
- fillna
- split
- impute mean, mode, median: SimpleImputer
- integer encoding: LabelEncoder
- one hot encoding: OneHotEncoder
- scale

In [21]:
df.drop(columns = ["deck"], inplace = True)

In [22]:
df.fillna(np.nan, inplace = True) # preprocess null into numpy.nan so imputer can process

In [23]:
import split_scale
train, test = train_test_split(df, train_size = 0.8, random_state = 123)

In [9]:
# create imputer object
# Impute mean, mode, median using SimpleImputer
# imp_mode = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imp_mode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

In [10]:
imp_mode.fit(train[['embarked']])

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='most_frequent', verbose=0)

In [11]:
train['embarked'] = imp_mode.transform(train[['embarked']])
test['embarked'] = imp_mode.transform(test[['embarked']])

In [12]:
test.embarked.value_counts(dropna = False)

S    129
C     40
Q     10
Name: embarked, dtype: int64

Encoding

1. integer encoding
1. one hot encoding

In [43]:
int_encoder = LabelEncoder()
int_encoder.fit(train.embarked)
#transformed data will be pandas series
train.embarked = int_encoder.transform(train.embarked)# need to impute, no missing value or messy data before encode
test.embarked = int_encoder.transform(test.embarked)

In [46]:
train.embarked.value_counts()

2    517
0    128
1     67
Name: embarked, dtype: int64

- Use int_encoder to transform categorical into integer
- but the integers are not "real number"
- process with "one hot encoding"
- need to convert into 2D array before sending 

In [50]:
embarked_array = np.array(train.embarked).reshape(len(train.embarked), 1)#become 2D array
embarked_array_test = np.array(test.embarked).reshape(len(test.embarked), 1)

In [55]:
# create onehotencoder object
ohe = OneHotEncoder(sparse=False, categories='auto') 
embarked_ohe = ohe.fit_transform(embarked_array)
embarked_ohe_test = ohe.transform(embarked_array_test)

In [17]:
embarked_ohe_test

array([[0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0

In [36]:
imp_mode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp_mode.fit(train[['embarked']])
train['embarked'] = imp_mode.transform(train[['embarked']])
test['embarked'] = imp_mode.transform(test[['embarked']])

In [37]:
train.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,embark_town,alone
329,329,1,1,female,16.0,0,1,57.9792,C,First,Cherbourg,0
749,749,0,3,male,31.0,0,0,7.7500,Q,Third,Queenstown,1
203,203,0,3,male,45.5,0,0,7.2250,C,Third,Cherbourg,1
421,421,0,3,male,21.0,0,0,7.7333,Q,Third,Queenstown,1
97,97,1,1,male,23.0,0,1,63.3583,C,First,Cherbourg,0


In [54]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder(categorical_features = train.embarked)
x = onehotencoder.fit_transform(train.embarked).toarray()

IndexError: tuple index out of range

use imputing instead of my fillna(median) method globally decrease the chance of overfitting
- generate impute object
- fit train with impute object
- transform train and test independently

sometimes you need to filter down instead of for example globally impute age as median(age)

**Encoding**
1. integer encoding
2. one hot encoding

In [ ]:
int_encoder = LabelEncoder()
int_encoder.fit(train.embarked) 
train.embarked_int = int_encoder.transform(train.embarked)
# this will be integer, will become prob

In [ ]:
embarked_array = np.array(train.embarked)
embarked_array = embarked_array.reshape(len(embarked_array), 1) 
ohe = OneHotEncoder(sparse = False, categories = 'auto')
ohe.fit_transform(embarked_array)


# sparse = sparse matrix or not

In [ ]:
df.isnull().sum()
df.embarked.value_counts(dropna=False) # will show na
# pandas profilling